<center>
    <h1>
            Probabilidad y Estadística
    </h1>
    <h2>
            Trabajo Práctico
    </h2>
</center>

<table>
<thead>
  <tr>
    <th>Integrante</th>
    <th>Legajo</th>
  </tr>
</thead>
<tbody>
  <tr>
    <td>Cintia Capece</td>
    <td>xxxxx</td>
  </tr>
  <tr>
    <td>Nicolás Bellesi</td>
    <td>xxxxx</td>
  </tr>
  <tr>
    <td>Lucas Nevado</td>
    <td>40768</td>
  </tr>
</tbody>
</table>

## Índice

* [Setup](#setup)
* [Parte 1: Simulación](#parte_1)
    * [1.1 Generador Bernoulli](#p_1_1)
    * [1.2 Generador Binomial](#p_1_2)
    * [1.3 Generador Exponencial](#p_1_3)
    * [1.4 Generador Normal (y Normal Estándar)](#p_1_4)
* [Parte 2: Estadística descriptiva](#parte_2)
    * [2.1 Muestras Exp(0.5)](#p_2_1)
    * [2.3 Histogramas Exp(0.5)](#p_2_2)
    * [2.3 Muestra n=50 Bin(10, 0.3) y función de distribución empírica](#p_2_3)
    * [2.4 Nueva muestra a partir de la función de distribución empírica. Media y varianza muestral](#p_2_4)
    * [2.5 Repetición del experimento](#p_2_5)
* [Parte 3: Convergencia](#parte_3)
    * [3.1 Histogramas Binomial p=0.40](#p_3_1)
    * [3.2 Normalización Bin(200, 0.40)](#p_3_2)
    * [3.3 Medias muestrales Binomial p=0.40](#p_3_3)
* [Parte 4: Estadística inferencial](#parte_4)
    * [4.1 Estimaciones puntuales N(100, 5)](#p_4_1)
    * [4.2 Intervalos de confianza con varianza conocida](#p_4_2)
    * [4.3 Intervalos de confianza con varianza desconocida](#p_4_3)
    * [4.4 Probabilidad de cometer un error de tipo II](#p_4_4)
    * [4.5 Muestra proveniente de una distribución normal](#p_4_5)
* [Bibliografía](#bibliografia)

---

## Setup<a class="anchor" id="setup"></a>

In [1]:
import random
import math
import numpy as np
import scipy.stats as st
from scipy.integrate import quad
from bokeh.io import output_notebook
from bokeh.plotting import figure, show
from bokeh.layouts import row
from bokeh.models import Legend, LegendItem


output_notebook()

def muestra_gen(muestra_tam, distribucion_gen, *distribucion_args):
    """
    muestra_tam: Tamaño de la muestra
    distribucion_gen: Generador de una distribución, ej: exponencial_gen
    distribucion_args: Argumentos necesarios para la distribución, ej: 0.5
    """
    muestra = []
    for i in range(muestra_tam):
        valor = distribucion_gen(*distribucion_args)
        muestra.append(valor)

    return muestra

def crear_histograma(muestra, alto=400, ancho=400, ancho_de_banda = 0, normalizado=True, titulo="", eje_x="", eje_y=""):
    """
    A partir de una muestra, devuelve una histograma de bokeh.
    Salvo la muestra, todos los demás parámetros son opcionales.
    """
    if ancho_de_banda == 0:
        hist, edges = np.histogram(muestra, density=normalizado)
    else:
        hist, edges = np.histogram(muestra, density=normalizado, bins=math.floor((max(muestra) - min(muestra)) / ancho_de_banda))

    p = figure(title=titulo, plot_height=alto, plot_width=ancho, x_axis_label=eje_x, y_axis_label=eje_y)
    p.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:], line_color="white")
    return p

def agregar_media(histograma, media):
    histograma.ray(x=[media], y=[0], length=0, angle=(np.pi) / 2, line_width=3, line_color="red", legend_label="x̄ = " + str(media))
    return histograma

def media_muestral(muestra):
    return sum(muestra) / len(muestra)

def varianza_muestral(muestra):
    media = media_muestral(muestra)
    return sum([(x - media) ** 2 for x in muestra]) / (len(muestra) - 1)
    

Loading BokehJS ...

## Parte 1: Simulación<a class="anchor" id="parte_1"></a>

### 1) Generador Bernoulli<a class="anchor" id="p_1_1"></a>

In [2]:
def bernoulli_gen(p):
    uniforme = random.uniform(0, 1)
    if uniforme < p:
        return 1
    else:
        return 0

In [3]:
bernoulli_gen(0.75)

0

### 2) Generador Binomial<a class="anchor" id="p_1_2"></a>

In [4]:
def binomial_gen(n, p):
    exitos = 0
    for i in range(n):
        exitos += bernoulli_gen(p)

    return exitos

In [5]:
binomial_gen(10, 0.5)

3

### 3) Generador Exponencial<a class="anchor" id="p_1_3"></a>

In [6]:
def exponencial_gen(λ):
    """
    Utilizamos el método de la función inversa
    """
    uniforme = random.uniform(0, 1)
    
    exponencial = - (1 / λ) * math.log(uniforme)

    return exponencial

In [7]:
exponencial_gen(3)

0.6205980258558702

### 4) Generador Normal (y Normal Estándar)<a class="anchor" id="p_1_4"></a>

In [8]:
def std_normal_gen():
    """
    Utilizamos el método de Box-Muller
    """
    uniforme_U = random.uniform(0, 1)
    uniforme_V = random.uniform(0, 1)

    std_normal = math.sqrt(-2 * math.log(uniforme_U)) * math.cos(2 * math.pi * uniforme_V)
    
    return std_normal

In [9]:
std_normal_gen()

-0.9819901422552955

In [10]:
def normal_gen(μ = 0, σ_2 = 1):
    σ = math.sqrt(σ_2)
    normal = std_normal_gen() * σ + μ

    return normal

In [11]:
normal_gen(1, 1)

0.6900032369309956

## Parte 2: Estadística descriptiva<a class="anchor" id="parte_2"></a>

### 1) Muestras Exp(0.5)<a class="anchor" id="p_2_1"></a>

In [12]:
exp_n_10 = muestra_gen(10, exponencial_gen, 0.5)
exp_n_30 = muestra_gen(30, exponencial_gen, 0.5)
exp_n_200 = muestra_gen(20000, exponencial_gen, 0.5)

print("Media Muestral Exp(0.5), n = 10: ", round(media_muestral(exp_n_10), 3))
print("Varianza Muestral Exp(0.5), n = 10: ", round(varianza_muestral(exp_n_10), 3))

print("Media Muestral Exp(0.5), n = 30: ", round(media_muestral(exp_n_30), 3))
print("Varianza Muestral Exp(0.5), n = 30: ", round(varianza_muestral(exp_n_30), 3))

print("Media Muestral Exp(0.5), n = 200: ", round(media_muestral(exp_n_200), 3))
print("Varianza Muestral Exp(0.5), n = 200: ", round(varianza_muestral(exp_n_200), 3))

Media Muestral Exp(0.5), n = 10:  1.948
Varianza Muestral Exp(0.5), n = 10:  3.931
Media Muestral Exp(0.5), n = 30:  1.72
Varianza Muestral Exp(0.5), n = 30:  4.154
Media Muestral Exp(0.5), n = 200:  2.008
Varianza Muestral Exp(0.5), n = 200:  4.081


La media y la varianza muestral se aproximan mejor a la media y varianza de la distribución con un mayor tamaño de muestra. Aunque, por el sesgo del estimador de la varianza, podemos llegar a tener valores dispares.

### 2) Histogramas Exp(0.5)<a class="anchor" id="p_2_2"></a>

In [13]:
hist_04 = crear_histograma(exp_n_10, ancho_de_banda=0.4, titulo="Exp(0.5), n = 10, ancho de banda = 0.4")
hist_02 = crear_histograma(exp_n_10, ancho_de_banda=0.2, titulo="Exp(0.5), n = 10, ancho de banda = 0.2")
hist_01 = crear_histograma(exp_n_10, ancho_de_banda=0.1, titulo="Exp(0.5), n = 10, ancho de banda = 0.1")
show(row(hist_04,hist_02,hist_01, sizing_mode='scale_both'))

hist_04 = crear_histograma(exp_n_30, ancho_de_banda=0.4, titulo="Exp(0.5), n = 30, ancho de banda = 0.4")
hist_02 = crear_histograma(exp_n_30, ancho_de_banda=0.2, titulo="Exp(0.5), n = 30, ancho de banda = 0.2")
hist_01 = crear_histograma(exp_n_30, ancho_de_banda=0.1, titulo="Exp(0.5), n = 30, ancho de banda = 0.1")
show(row(hist_04,hist_02,hist_01, sizing_mode='scale_both'))

hist_04 = crear_histograma(exp_n_200, ancho_de_banda=0.4, titulo="Exp(0.5), n = 200, ancho de banda = 0.4")
hist_02 = crear_histograma(exp_n_200, ancho_de_banda=0.2, titulo="Exp(0.5), n = 200, ancho de banda = 0.2")
hist_01 = crear_histograma(exp_n_200, ancho_de_banda=0.1, titulo="Exp(0.5), n = 200, ancho de banda = 0.1")
show(row(hist_04,hist_02,hist_01, sizing_mode='scale_both'))

Se pueden observar dos cosas a partir de los histogramas:

En primera instancia, por su asimetría, muestras con distribuciones exponencial necesitan ser de un gran tamaño para notar en un histograma la forma correcta.

Por otro lado, también es importante el ancho de banda que se utiliza al graficar el histograma. Un ancho de banda mayor produce una menor cantidad de barras (y viceversa), se debe encontrar un ancho de banda óptimo que permita ver en el histograma información interesante como la forma, las modas y la simetría.

### 3) Muestra n=50 Bin(10, 0.3) y función de distribución empírica<a class="anchor" id="p_2_3"></a>

In [14]:
muestra_bin = muestra_gen(50, binomial_gen, 10, 0.3)

def fde(muestra, x):
    muestra_ordenada = np.sort(muestra)
    N = len(muestra)
    return np.searchsorted(muestra_ordenada, x) / N

fde(muestra_bin, 3)

0.36

### 4) Nueva muestra a partir de la función de distribución empírica. Media y varianza muestral<a class="anchor" id="p_2_4"></a>

In [15]:
def muestra_fde(muestra):
    muestra_fde = []
    for i in muestra_bin:
        v = fde(muestra_bin, i)
        muestra_fde.append(v)
    return muestra_fde

muestra_fde_bin = muestra_fde(muestra_bin)

media_muestral_bin = round(media_muestral(muestra_bin), 3)
media_muestral_fde_bin = round(media_muestral(muestra_fde_bin), 3)
varianza_muestral_bin = round(varianza_muestral(muestra_bin), 3)
varianza_muestral_fde_bin = round(varianza_muestral(muestra_fde_bin), 3)

print("Media Muestral Bin(10, 0.3): ", media_muestral_bin)
print("Media Muestral FDE(Bin(10, 0.3)): ", media_muestral_fde_bin)

print("Varianza Muestral Bin(10, 0.3): ", varianza_muestral_bin)
print("Varianza Muestral FDE(Bin(10, 0.3)): ", varianza_muestral_fde_bin)

hist1 = crear_histograma(muestra_bin, titulo="Bin(10, 0.3), n=50")
agregar_media(hist1, media_muestral_bin)

hist2 = crear_histograma(muestra_fde_bin, titulo="FDE(Bin(10, 0.3)), n=50")
agregar_media(hist2, media_muestral_fde_bin)

show(row(hist1, hist2, sizing_mode='scale_both'))

Media Muestral Bin(10, 0.3):  3.16
Media Muestral FDE(Bin(10, 0.3)):  0.396
Varianza Muestral Bin(10, 0.3):  2.096
Varianza Muestral FDE(Bin(10, 0.3)):  0.093


### 5) Repetición del experimento<a class="anchor" id="p_2_5"></a>

In [16]:
# Primera repetición
muestra_bin = muestra_gen(50, binomial_gen, 10, 0.3)
muestra_fde_bin = muestra_fde(muestra_bin)


media_muestral_bin = round(media_muestral(muestra_bin), 3)
media_muestral_fde_bin = round(media_muestral(muestra_fde_bin), 3)
varianza_muestral_bin = round(varianza_muestral(muestra_bin), 3)
varianza_muestral_fde_bin = round(varianza_muestral(muestra_fde_bin), 3)

print("2 - Media Muestral Bin(10, 0.3): ", media_muestral_bin)
print("2 - Media Muestral FDE(Bin(10, 0.3)): ", media_muestral_fde_bin)

print("2 - Varianza Muestral Bin(10, 0.3): ", varianza_muestral_bin)
print("2 - Varianza Muestral FDE(Bin(10, 0.3)): ", varianza_muestral_fde_bin)

hist1 = crear_histograma(muestra_bin, titulo="Bin(10, 0.3), n=50")
agregar_media(hist1, media_muestral_bin)

hist2 = crear_histograma(muestra_fde_bin, titulo="FDE(Bin(10, 0.3)), n=50")
agregar_media(hist2, media_muestral_fde_bin)

show(row(hist1, hist2, sizing_mode='scale_both'))

# Segunda repetición
muestra_bin = muestra_gen(50, binomial_gen, 10, 0.3)
muestra_fde_bin = muestra_fde(muestra_bin)

media_muestral_bin = round(media_muestral(muestra_bin), 3)
media_muestral_fde_bin = round(media_muestral(muestra_fde_bin), 3)
varianza_muestral_bin = round(varianza_muestral(muestra_bin), 3)
varianza_muestral_fde_bin = round(varianza_muestral(muestra_fde_bin), 3)

print("3 - Media Muestral Bin(10, 0.3): ", media_muestral_bin)
print("3 - Media Muestral FDE(Bin(10, 0.3)): ", media_muestral_fde_bin)

print("3 - Varianza Muestral Bin(10, 0.3): ", varianza_muestral_bin)
print("3 - Varianza Muestral FDE(Bin(10, 0.3)): ", varianza_muestral_fde_bin)

hist1 = crear_histograma(muestra_bin, titulo="Bin(10, 0.3), n=50")
agregar_media(hist1, media_muestral_bin)

hist2 = crear_histograma(muestra_fde_bin, titulo="FDE(Bin(10, 0.3)), n=50")
agregar_media(hist2, media_muestral_fde_bin)

show(row(hist1, hist2, sizing_mode='scale_both'))

2 - Media Muestral Bin(10, 0.3):  3.0
2 - Media Muestral FDE(Bin(10, 0.3)):  0.397
2 - Varianza Muestral Bin(10, 0.3):  2.408
2 - Varianza Muestral FDE(Bin(10, 0.3)):  0.088


3 - Media Muestral Bin(10, 0.3):  2.7
3 - Media Muestral FDE(Bin(10, 0.3)):  0.41
3 - Varianza Muestral Bin(10, 0.3):  2.622
3 - Varianza Muestral FDE(Bin(10, 0.3)):  0.085


## Parte 3: Convergencia<a class="anchor" id="parte_3"></a>

### 1) Histogramas Binomial p=0.40<a class="anchor" id="p_3_1"></a>

In [17]:
muestra_n10 = muestra_gen(100,binomial_gen,10,0.4);
muestra_n20 = muestra_gen(100, binomial_gen, 20, 0.4);
muestra_n50 = muestra_gen(100, binomial_gen, 50, 0.4);
muestra_n100 = muestra_gen(100, binomial_gen, 100, 0.4);

hist1 = crear_histograma(muestra_n10, titulo="Bin(10, 0.4)");
hist2 = crear_histograma(muestra_n20, titulo="Bin(20, 0.4)");
hist3 = crear_histograma(muestra_n50, titulo="Bin(50, 0.4)");
hist4 = crear_histograma(muestra_n100, titulo="Bin(100, 0.4)");
show(row(hist1, hist2, hist3, hist4, sizing_mode='scale_both'))

Observando los histogramas podemos notar que a medida que crece el $n$ de las distribuciones binomiales, las muestras generadas con dicha distribución van tomando la forma de una distribución normal, esto quiere decir que la distribución normal puede aproximarse por la binomial, aunque no exactamente, debido a la carácter discreto de esta última.

Además, las muestras generadas tienen su valor medio en el punto más probable, por lo tanto se deduce que la media es $\mu = n \cdot p$.

### 2) Normalización Bin(200, 0.40)<a class="anchor" id="p_3_2"></a>

In [18]:
def normalizar_elementos_de_muestra(muestra):

    lista_normalizada = [];
    mediamuestral = media_muestral(muestra);
    desviacion_muestral = math.sqrt(varianza_muestral(muestra));
    for x in range(0,len(muestra)):
        normal = (muestra[x]-mediamuestral)/desviacion_muestral;
        lista_normalizada.append(normal);
    return lista_normalizada;

normaln100 = normalizar_elementos_de_muestra(muestra_n100);

print("Media Muestral Bin(100, 0.40): ", round(media_muestral(muestra_n100), 3))
print("Desviación Estándar Muestral Bin(100, 0.40): ", round(math.sqrt(varianza_muestral(muestra_n100)), 3))

hist1 = crear_histograma(normaln100, titulo="Bin(100, 0.4) normalizada");
show(hist1)

Media Muestral Bin(100, 0.40):  39.9
Desviación Estándar Muestral Bin(100, 0.40):  4.939


Para este gráfico, observamos que tiene la forma de una distribución $N(0, 1)$. Esto se explica con el Teorema de DeMoivre-Laplace, el cual es un caso especial del Teorema del Límite Central y dice que:

Sean $X_1, X_2, ..., X_n$ variables aleatorias independientes e idénticamente distribuidas, entonces: $\frac{S_n - \mu}{\sigma} \rightarrow Z \sim N(0, 1)$, cuando $n \rightarrow +\infty$.

Donde $S_n$ es una sucesión de variables aleatorias con distribución binomial. 

Esto quiere decir que al normalizar las muestras, convergen en distribución a una normal con media 0 y desviación estándar 1 cuando $n$ se hace cada vez más grande.

En el caso del ejercicio, usamos los estadísticos media muestral y varianza muestral como $\mu$ y $\sigma$ respectivamente y no trabajamos con la sucesión ya que normalizamos cada dato por separado para finalmente ver cómo converge a una $N(0, 1)$ .
El gráfico será cada vez más parecido y preciso respecto a una Normal cuando se toman valores de $n$ grandes, como por ejemplo 100000

### 3) Medias muestrales Binomial p=0.40<a class="anchor" id="p_3_3"></a>

In [19]:
print("Media Muestral Bin(10, 0.40): ", round(media_muestral(muestra_n10), 3))
print("Media Muestral Bin(20, 0.40): ", round(media_muestral(muestra_n20), 3))
print("Media Muestral Bin(50, 0.40): ", round(media_muestral(muestra_n50), 3))
print("Media Muestral Bin(100, 0.40): ", round(media_muestral(muestra_n100), 3))

Media Muestral Bin(10, 0.40):  3.99
Media Muestral Bin(20, 0.40):  7.95
Media Muestral Bin(50, 0.40):  20.2
Media Muestral Bin(100, 0.40):  39.9


Calculando las medias muestrales podemos notar que se aproximan bastante a las medias poblacionales $n \cdot p$. Esto se debe a que si se toman muestras de una población y se aplica el estadístico $\overline{X} = \frac{1}{n} \cdot \sum_{i = 1}^{n} X_i$ para la media muestral, se tiene una buena estimación de la media poblacional.

## Parte 4: Estadística inferencial<a class="anchor" id="parte_4"></a>

### 1) Estimaciones puntuales N(100, 5)<a class="anchor" id="p_4_1"></a>

In [20]:
muestra_N_100_5_10 = muestra_gen(10, normal_gen, 100, 5)
muestra_N_100_5_30 = muestra_gen(30, normal_gen, 100, 5)

print("Media Muestral N(100, 5), n=10: ", round(media_muestral(muestra_N_100_5_10), 3))
print("Media Muestral N(100, 5), n=30: ", round(media_muestral(muestra_N_100_5_30), 3))

print("Varianza Muestral N(100, 5), n=10: ", round(varianza_muestral(muestra_N_100_5_10), 3))
print("Varianza Muestral N(100, 5), n=30: ", round(varianza_muestral(muestra_N_100_5_30), 3))

Media Muestral N(100, 5), n=10:  100.211
Media Muestral N(100, 5), n=30:  100.466
Varianza Muestral N(100, 5), n=10:  10.212
Varianza Muestral N(100, 5), n=30:  3.551


### 2) Intervalos de confianza con varianza conocida<a class="anchor" id="p_4_2"></a>

In [21]:
def intervalo_de_confianza_mu_varianza_conocida(nivel_porcentaje, muestra, varianza):
    p = 1 - ( (100 - nivel_porcentaje) / (100 * 2) ) 
    z = st.norm.ppf(p)
    n = len(muestra)
    σ = math.sqrt(varianza)
    limite_inferior = round(media_muestral(muestra) - z * (σ / math.sqrt(n)), 3)
    limite_superior = round(media_muestral(muestra) + z * (σ / math.sqrt(n)), 3)
    return [limite_inferior, limite_superior]

print("Intervalo de confianza del 95%, N(100, 5), n=10: ", intervalo_de_confianza_mu_varianza_conocida(95, muestra_N_100_5_10, 5))
print("Intervalo de confianza del 98%, N(100, 5), n=10: ", intervalo_de_confianza_mu_varianza_conocida(98, muestra_N_100_5_10, 5))

print("Intervalo de confianza del 95%, N(100, 5), n=30: ", intervalo_de_confianza_mu_varianza_conocida(95, muestra_N_100_5_30, 5))
print("Intervalo de confianza del 98%, N(100, 5), n=30: ", intervalo_de_confianza_mu_varianza_conocida(98, muestra_N_100_5_30, 5))

Intervalo de confianza del 95%, N(100, 5), n=10:  [98.825, 101.597]
Intervalo de confianza del 98%, N(100, 5), n=10:  [98.566, 101.856]
Intervalo de confianza del 95%, N(100, 5), n=30:  [99.666, 101.266]
Intervalo de confianza del 98%, N(100, 5), n=30:  [99.517, 101.416]


### 3) Intervalos de confianza con varianza desconocida<a class="anchor" id="p_4_3"></a>

In [22]:
def intervalo_de_confianza_mu_varianza_desconocida(nivel_porcentaje, muestra):
    n = len(muestra)
    p = 1 - ( (100 - nivel_porcentaje) / (100 * 2) )
    t = st.t.ppf(p, n - 1)
    S = math.sqrt(varianza_muestral(muestra))
    limite_inferior = round(media_muestral(muestra) - t * (S / math.sqrt(n)), 3)
    limite_superior = round(media_muestral(muestra) + t * (S / math.sqrt(n)), 3)
    return [limite_inferior, limite_superior]

print("Intervalo de confianza del 95%, N(100, 5), n=10: ", intervalo_de_confianza_mu_varianza_desconocida(95, muestra_N_100_5_10))
print("Intervalo de confianza del 98%, N(100, 5), n=10: ", intervalo_de_confianza_mu_varianza_desconocida(98, muestra_N_100_5_10))

print("Intervalo de confianza del 95% N(100, 5), n=30: ", intervalo_de_confianza_mu_varianza_desconocida(95, muestra_N_100_5_30))
print("Intervalo de confianza del 98% N(100, 5), n=30: ", intervalo_de_confianza_mu_varianza_desconocida(98, muestra_N_100_5_30))

Intervalo de confianza del 95%, N(100, 5), n=10:  [97.925, 102.497]
Intervalo de confianza del 98%, N(100, 5), n=10:  [97.36, 103.062]
Intervalo de confianza del 95% N(100, 5), n=30:  [99.763, 101.17]
Intervalo de confianza del 98% N(100, 5), n=30:  [99.619, 101.313]


### 4) Probabilidad de cometer un error de tipo II<a class="anchor" id="p_4_4"></a>

In [23]:
muestra_n30 = muestra_gen(30, normal_gen, 100, 5)
n = len(muestra_n30)
chi = st.chi2.ppf(0.99, n - 1)
varmuestral = varianza_muestral(muestra_n30);
x_obs = ( (n-1) * varmuestral ) / 5;
print("La varianza muestral es: ", round(varmuestral, 3));
print("Prueba de hipotesis con nivel de significancia 0.01");
print("H0: la varianza es mayor a 5\nH1: La varianza es menor que 5\n");
print("El valor crítico es: ", round(chi, 3));
print("El valor X observado en distribucion chi cuadrado es X= ", round(x_obs, 3), "\n");
print("Como el X observado es menor que el valor crítico, se rechaza H0\n");

print("Probabilidad de cometer error tipo 2 si la varianza era 6");
varianza_limite = (chi * 5) / (n-1); #despejo la varianza limite para reemplazar en el estadistico y sacar el nuevo valor critico
nuevo_x_critico = ( (n-1) * varianza_limite ) / 6; #busco el nuevo valor critico X con distribucion chi cuadrado
p = st.chi2.sf(nuevo_x_critico, n - 1)
print("Ya que el error de tipo II es aceptar H0 cuando era falsa, entonces busco la P(X >", round(nuevo_x_critico, 3), "), es decir, el nuevo valor critico.\nPor la tabla de chi cuadrado, esa probabilidad da aprox: ", round(p, 3), " o ", round(p, 3) * 100, "%");

La varianza muestral es:  3.794
Prueba de hipotesis con nivel de significancia 0.01
H0: la varianza es mayor a 5
H1: La varianza es menor que 5

El valor crítico es:  49.588
El valor X observado en distribucion chi cuadrado es X=  22.003 

Como el X observado es menor que el valor crítico, se rechaza H0

Probabilidad de cometer error tipo 2 si la varianza era 6
Ya que el error de tipo II es aceptar H0 cuando era falsa, entonces busco la P(X > 41.323 ), es decir, el nuevo valor critico.
Por la tabla de chi cuadrado, esa probabilidad da aprox:  0.065  o  6.5 %


### 5) Muestra proveniente de una distribución normal<a class="anchor" id="p_4_5"></a>

In [24]:
def estandarizar(x,media,desviacion):
    return (x-media)/desviacion;

def calculadora_normal(z):
    inf = -np.inf
    cte = 1/math.sqrt(2*math.pi);
    fx = lambda x: cte*pow(math.e,-(x**2)/2);
    resultado = quad(fx,inf,z)
    return resultado;

muestra_30 = muestra_gen(30, normal_gen, 100, 5)
n = len(muestra_30)
minimo = min(muestra_30);
maximo = max(muestra_30);


## LIMITES
limites = [[-1 for i in range(2)] for j in range(25)] #COLUMNAS Y FILAS
acum = minimo;
i=0;
while(maximo>=acum):
    limites[i][0]=acum;
    acum+=0.5;
    limites[i][1]=acum;
    i += 1;

##ESPERADOS:

i=0
observados=[];

while (limites[i][0] != -1):
    contador = 0;
    for pos in range(0,len(muestra_30)):
        if(muestra_30[pos]>limites[i][0] and muestra_30[pos]<limites[i][1]):
            contador+=1
    observados.append(contador);
    i+=1;


#PROBABILIDADES
a=0
probabilidades=[]
while(limites[a][0]!=-1):
    z1 = estandarizar(limites[a][0],100,math.sqrt(5));
    z2 = estandarizar(limites[a][1],100,math.sqrt(5));

    proba = calculadora_normal(z2)[0]-calculadora_normal(z1)[0]
    probabilidades.append(proba);
    a+=1

#ESTADISTICO
estadisticos=[];
for k in range(0,len(observados)):
    esperado = probabilidades[k]*30;
    est = pow(observados[k]-esperado,2)/esperado
    estadisticos.append(est);

chi = sum(estadisticos);

if(chi < 49.58):
    print("Como el estadístico de prueba ", round(chi, 3), " es menor al estadístico teórico 49.58, se distribuye normalmente");
else:
    print("Como el estadístico de prueba ", round(chi, 3), " es mayor al estadístico teórico 49.58, no se distribuye normalmente");

Como el estadístico de prueba  21.774  es menor al estadístico teórico 49.58, se distribuye normalmente


## Bibliografía<a class="anchor" id="bibliografia"></a>